In [2]:
import warnings
warnings.filterwarnings( 'ignore' )

In [3]:
import pandas as pd
import numpy as np
import os
if os.name != 'posix':
    plt.rc("font", family="Malgun Gothic")
from sklearn.model_selection import TimeSeriesSplit
from xgboost import XGBClassifier
from xgboost import XGBRegressor
from sklearn.model_selection import train_test_split
from sklearn.metrics import f1_score
from sklearn.metrics import multilabel_confusion_matrix
from sklearn.metrics import confusion_matrix, accuracy_score 
from sklearn.utils import class_weight

In [4]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [29]:
sk=pd.read_csv('/content/SK_label(5일추가).csv')

In [9]:
label=pd.DataFrame()

#encoding변수 추가
label['encoding'] = sk['5일 등락률'].apply(lambda x: 1 if abs(x) < 5 else 0 if x >= 5 else 2)

In [10]:
sk1=sk.shift(1) # 한 행씩 데이터 아래로 내리기
df=pd.concat([sk1,label],axis=1)
df=df.drop(0,axis=0)
df=df.reset_index(drop=True)

In [12]:
df1=df.copy()

In [14]:
#KS검정을 통해 선택한 변수 list에 저장.
cor_list=['등락률','종가','거래량','거래대금','시가총액','토론방','순매수_기관','순매수_기타법인','순매수_개인','순매수_외국인','검색어','보도량','기사감성점수','코스피등락률','코스피거래대금'] #3일치


In [15]:
#x, y 지정
x=df1[cor_list]
y=df1['encoding']

#분류(XGB,LGB)

In [17]:
tscv=TimeSeriesSplit(n_splits=4) 

for i, (train_index, test_index) in enumerate(tscv.split(x)):
    x_train, y_train=x.iloc[train_index], y.iloc[train_index]
    x_test, y_test=x.iloc[test_index],y.iloc[test_index]
    classes_weights = class_weight.compute_sample_weight(
    class_weight='balanced',y=y_train)  #클래스 불균형 개선   
    params = {'objective': 'multi:softmax', 'num_class': 3}
    model = XGBClassifier(**params).fit(x_train, y_train,sample_weight=classes_weights)
    model.fit(x_train, y_train, sample_weight=classes_weights)
    
    y_pred=model.predict(x_test)
    print(f"Fold {i+1} - accuracy: {accuracy_score(y_test, y_pred)}")
    print(f"Fold {i+1} - f1_score: {f1_score(y_test,y_pred,average='weighted')}")

Fold 1 - accuracy: 0.6761565836298933
Fold 1 - f1_score: 0.6028224864545777
Fold 2 - accuracy: 0.7402135231316725
Fold 2 - f1_score: 0.7068984186801797
Fold 3 - accuracy: 0.5729537366548043
Fold 3 - f1_score: 0.6008356059694371
Fold 4 - accuracy: 0.697508896797153
Fold 4 - f1_score: 0.6603275239707725


In [20]:
cm = confusion_matrix(y_test, y_pred)

print(cm)
print()
print('전체 정확도 : ', accuracy_score(y_test, y_pred))
print('전체 f1-score : ', f1_score(y_test, y_pred, average='weighted'))
print()
print('매수 정확도 : ', cm[0,0]/sum(cm[0]))
print('매도 정확도 : ', cm[2,2]/sum(cm[2]))
print('유지 정확도 : ', cm[1,1]/sum(cm[1]))
print()
print('매수 정밀도 : ', cm[0,0]/np.sum(cm, axis=0)[0])
print('매도 정밀도 : ', cm[2,2]/np.sum(cm, axis=0)[2])

[[  6  31   0]
 [ 10 177  11]
 [  2  31  13]]

전체 정확도 :  0.697508896797153
전체 f1-score :  0.6603275239707725

매수 정확도 :  0.16216216216216217
매도 정확도 :  0.2826086956521739
유지 정확도 :  0.8939393939393939

매수 정밀도 :  0.3333333333333333
매도 정밀도 :  0.5416666666666666


In [21]:
!pip install optuna

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 365.7/365.7 kB 28.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 212.9/212.9 kB 25.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 78.7/78.7 kB 11.2 MB/s eta 0:00:00


#optuna

In [24]:
import optuna
from sklearn.metrics import f1_score
import xgboost as xgb
from xgboost import XGBClassifier
from sklearn.model_selection import TimeSeriesSplit

tscv = TimeSeriesSplit(n_splits=5)

def objective(trial):
    param={
        'max_depth': trial.suggest_int('max_depth', 1, 20),
        'learning_rate': trial.suggest_loguniform('learning_rate', 0.001, 1.0),
        'n_estimators': trial.suggest_int('n_estimators', 10, 1000),
        'min_child_weight': trial.suggest_int('min_child_weight', 1, 10),
        'gamma': trial.suggest_loguniform('gamma', 1e-8, 1.0),
        'subsample': trial.suggest_loguniform('subsample', 1e-8, 1.0),
        'colsample_bytree': trial.suggest_loguniform('colsample_bytree', 1e-8, 1.0),
        'reg_alpha': trial.suggest_loguniform('reg_alpha', 1e-8, 1.0),
        'reg_lambda': trial.suggest_loguniform('reg_lambda', 1e-8, 1.0)
    } #optuna를 위한 파라미터 지정

    scores = [] #f1 score 저장 리스트 생성
    for i, (train_index, test_index) in enumerate(tscv.split(x)):
        x_train, y_train = x.iloc[train_index],y.iloc[train_index] #train data 지정
        x_test, y_test = x.iloc[test_index],y.iloc[test_index] # test data 지정

        classes_weights = class_weight.compute_sample_weight(class_weight='balanced',y=y_train) #클래스 불균형 개선  

        xgb_model = xgb.XGBClassifier(**param, random_state = 42)
        xgb_model.fit(x_train, y_train,sample_weight=classes_weights)
        
        y_pred = xgb_model.predict(x_test) #예측 
        score = f1_score(y_test, y_pred, average='weighted')
        scores.append(score)
    
    
    cm = confusion_matrix(y_test, y_pred)
    bacc = cm[0,0]/sum(cm[0]) #매수 정확도
    sacc = cm[2,2]/sum(cm[2]) #매도 정확도

    if np.sum(cm, axis=0)[0] == 0: #매수 정밀도
        bpre = 0
    else:
        bpre = cm[0,0]/np.sum(cm, axis=0)[0]

    if np.sum(cm, axis=0)[2] == 0: #매도 정밀도
        spre = 0
    else:
        spre = cm[2,2]/np.sum(cm, axis=0)[2]

    rst = np.mean([bacc, sacc, bpre, spre]) #매수 정확도, 매도 정확도, 매수 정밀도, 매도 정밀도 평균 저장 


    return rst

In [25]:
study = optuna.create_study(direction='maximize')
study.optimize(objective, n_trials=200)

[I 2023-05-09 07:49:01,805] A new study created in memory with name: no-name-5ca4c5f5-88c5-493b-acd1-6998605634ce
[I 2023-05-09 07:49:02,166] Trial 0 finished with value: 0.2841880341880342 and parameters: {'max_depth': 12, 'learning_rate': 0.23079709498704343, 'n_estimators': 101, 'min_child_weight': 2, 'gamma': 2.2421546311921226e-05, 'subsample': 0.0010917929604104463, 'colsample_bytree': 8.05118441584842e-05, 'reg_alpha': 1.8334232362930453e-06, 'reg_lambda': 3.8300078060441575e-08}. Best is trial 0 with value: 0.2841880341880342.
[I 2023-05-09 07:49:07,488] Trial 1 finished with value: 0.41010491338112304 and parameters: {'max_depth': 5, 'learning_rate': 0.0035399164930560666, 'n_estimators': 452, 'min_child_weight': 3, 'gamma': 8.550210148225183e-07, 'subsample': 0.37299953788308793, 'colsample_bytree': 1.6882455906544496e-06, 'reg_alpha': 0.02812156598259018, 'reg_lambda': 3.4081544894954983e-07}. Best is trial 1 with value: 0.41010491338112304.
[I 2023-05-09 07:49:09,072] Trial

In [26]:
# 학습 결과 출력
print(study.best_value)
print(study.best_params)

0.5551666719907998
{'max_depth': 9, 'learning_rate': 0.014197461926482859, 'n_estimators': 539, 'min_child_weight': 9, 'gamma': 7.702366029175775e-07, 'subsample': 0.11283412694057855, 'colsample_bytree': 0.9884290372881207, 'reg_alpha': 1.0081520700930986e-06, 'reg_lambda': 1.1744071257768952e-05}


In [27]:
tscv=TimeSeriesSplit()
TimeSeriesSplit(n_splits=4)

for i, (train_index, test_index) in enumerate(tscv.split(x)):
    x_train, y_train = x.iloc[train_index], y.iloc[train_index] #train data 지정
    x_test, y_test = x.iloc[test_index], y.iloc[test_index] # test data 지정

    classes_weights = class_weight.compute_sample_weight(
        class_weight='balanced', y=y_train) # 클래스 불균형 개선
    
    xgb_model=xgb.XGBClassifier(**study.best_params, random_state = 42)
    xgb_model.fit(x_train, y_train, sample_weight=classes_weights)

    y_pred = xgb_model.predict(x_test)
    print(f"Fold {i+1} - Accuracy: {accuracy_score(y_test, y_pred)}")
    print(f"Fold {i+1} - f1_score: {f1_score(y_test, y_pred, average='weighted')}")
    print()

Fold 1 - Accuracy: 0.44017094017094016
Fold 1 - f1_score: 0.44331087306361416

Fold 2 - Accuracy: 0.5683760683760684
Fold 2 - f1_score: 0.5953138521778183

Fold 3 - Accuracy: 0.48717948717948717
Fold 3 - f1_score: 0.532589289512993

Fold 4 - Accuracy: 0.47435897435897434
Fold 4 - f1_score: 0.5297278337599913

Fold 5 - Accuracy: 0.7008547008547008
Fold 5 - f1_score: 0.7115495645823515



In [28]:
cm = confusion_matrix(y_test, y_pred)

print(cm)
print()
print('전체 정확도 : ', accuracy_score(y_test, y_pred))
print('전체 f1-score : ', f1_score(y_test, y_pred, average='weighted'))
print()
print('매수 정확도 : ', cm[0,0]/sum(cm[0]))
print('매도 정확도 : ', cm[2,2]/sum(cm[2]))
print('유지 정확도 : ', cm[1,1]/sum(cm[1]))
print()
print('매수 정밀도 : ', cm[0,0]/np.sum(cm, axis=0)[0])
print('매도 정밀도 : ', cm[2,2]/np.sum(cm, axis=0)[2])

[[ 23   7   2]
 [ 17 120  26]
 [  3  15  21]]

전체 정확도 :  0.7008547008547008
전체 f1-score :  0.7115495645823515

매수 정확도 :  0.71875
매도 정확도 :  0.5384615384615384
유지 정확도 :  0.7361963190184049

매수 정밀도 :  0.5348837209302325
매도 정밀도 :  0.42857142857142855
